In [19]:
import glob
import os
import numpy as np
import rasterio
from rasterio.mask import mask
import geopandas as gpd

# -------------------------------------------------
# INPUT PATHS
los_dir = r"D:\ASF_InSAR_on_Demand\2025\Extracted\LOS_clipped"
aoi_shp = r"D:\ASF_InSAR_on_Demand\GMDA_Boundary\GMDA.shp"

# IMPORTANT: output must be a FILE, not a folder
output_tif = r"D:\ASF_InSAR_on_Demand\Time series\Results\LOS_MEAN_ALL_2025.tif"
# -------------------------------------------------

# Load AOI
aoi = gpd.read_file(aoi_shp)

# Find all LOS rasters
los_files = sorted(glob.glob(os.path.join(los_dir, "*.tif")))
print("Total LOS files found:", len(los_files))

los_stack = []
ref_meta = None

for f in los_files:
    with rasterio.open(f) as src:

        # Reproject AOI to raster CRS (CRITICAL)
        if aoi.crs != src.crs:
            aoi_proj = aoi.to_crs(src.crs)
        else:
            aoi_proj = aoi

        # Clip raster by AOI
        try:
            clipped, transform = mask(
                src,
                aoi_proj.geometry,
                crop=True,
                nodata=np.nan
            )
        except ValueError:
            print(f"Skipping {os.path.basename(f)}: no overlap")
            continue

        data = clipped[0].astype("float32")
        data[data == src.nodata] = np.nan

        if np.all(np.isnan(data)):
            print(f"Skipping {os.path.basename(f)}: empty after clip")
            continue

        los_stack.append(data)

        if ref_meta is None:
            ref_meta = src.meta.copy()
            ref_meta.update({
                "height": data.shape[0],
                "width": data.shape[1],
                "transform": transform,
                "dtype": "float32",
                "count": 1,
                "nodata": np.nan,
                "compress": "lzw"
            })

print("Valid rasters used:", len(los_stack))

# -------------------------------------------------
# PIXEL-WISE MEAN (AGGREGATE OF ALL IMAGES)
los_stack = np.stack(los_stack, axis=0)
mean_los = np.nanmean(los_stack, axis=0)

print("Valid pixels:", np.sum(~np.isnan(mean_los)))
print("LOS range (m):", np.nanmin(mean_los), "to", np.nanmax(mean_los))

# -------------------------------------------------
# WRITE OUTPUT GeoTIFF
with rasterio.open(output_tif, "w", **ref_meta) as dst:
    dst.write(mean_los, 1)

print("FINAL MEAN LOS GeoTIFF saved:")
print(output_tif)


Total LOS files found: 27
Valid rasters used: 27
Valid pixels: 234100
LOS range (m): -0.13982032 to 0.21100003
FINAL MEAN LOS GeoTIFF saved:
D:\ASF_InSAR_on_Demand\Time series\Results\LOS_MEAN_ALL_2025.tif


C:\Users\Standard\AppData\Local\Temp\ipykernel_9588\4191529626.py:74: RuntimeWarning: Mean of empty slice
  mean_los = np.nanmean(los_stack, axis=0)
